In [1]:
from matplotlib import pyplot as plt
from scipy import optimize
from numpy.linalg import norm
import circle_fit as cf
import numpy as np
import time
import math
import random
import sys
import cv2
import os
%run ../functions.ipynb

In [2]:
def calc_R(x,y, xc, yc):
    """
    calculate the distance of each 2D points from the center (xc, yc)
    """
    return np.sqrt((x-xc)**2 + (y-yc)**2)


def f(c, x, y, w):
    """
    calculate the algebraic distance between the data points
    and the mean circle centered at c=(xc, yc)
    """

    Ri = calc_R(x, y, *c)
    return w*(Ri - Ri.mean())
    

def LS_circle(coords, weights):
    """
    Circle fit using least-squares solver.
    Inputs:

        - coords, list or numpy array with len>2 of the form:
        [
    [x_coord, y_coord],
    ...,
    [x_coord, y_coord]
    ]
        or numpy array of shape (n, 2)

    Outputs:

        - xc : x-coordinate of solution center (float)
        - yc : y-coordinate of solution center (float)
        - R : Radius of solution (float)
        - residu : MSE of solution against training data (float)
    """

    x, y = None, None
    if isinstance(coords, np.ndarray):
        x = coords[:, 0]
        y = coords[:, 1]
    elif isinstance(coords, list):
        x = np.array([point[0] for point in coords])
        y = np.array([point[1] for point in coords])
    else:
        raise Exception("Parameter 'coords' is an unsupported type: " + str(type(coords)))

    # coordinates of the barycenter
    x_m = np.mean(x)
    y_m = np.mean(y)
    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f, center_estimate, args=(x,y,weights))
    xc, yc = center
    Ri       = calc_R(x, y, *center)
    R        = Ri.mean()
    residu   = np.sum((Ri - R)**2)
    return [xc, yc, R, residu]

## Get center/centroid functions

In [3]:
def get_centroid(X):
    return np.mean(X, axis=0)

def get_ls_center(X):
    cx, cy, r, _ = cf.least_squares_circle(X)
    return np.asarry([cx, cy])

def get_hyper_center(X, weights):
    cx, cy, r, _ = cf.hyper_fit(X)
    return np.asarry([cx, cy])

In [4]:
def get_symmetry_distances(X, params ,weights):
    # Step1. 取得當次圓心位置 及 初始化計算變數
    
    # 1-1 初始化變數
    size = X.shape[0]
    center = np.transpose([params[0], params[1]])
    
    symmetric_points  = set()
    SDs = list()
    # Step2. 計算平均對稱距離誤差（SDn）
    for i, xi in enumerate(X):
        
        # 2-1 初始當點最佳距離變數 及 相對應對稱點之索引
        current_distance = sys.maxsize
        symmetric_point_index   = 0
        for j, xj in enumerate(X):
            
            # 2-2 對於非自點 且 尚未被挑選當對稱點的資料都求對稱距離誤差 (SD)
            if (i != j & j not in symmetric_points):
                tmp = abs(norm(xi + xj - 2*center)) / (abs(norm(xi-center)) + abs(norm(xj-center)))
                if tmp < current_distance:
                    current_distance = tmp
                    symmetric_point_index = j
                    
        ## 2-3 類計誤差 並 紀錄以被使用過得點
        SDs.append(current_distance)
        symmetric_points.add(symmetric_point_index)
    
    return SDs

In [5]:
def get_weights(mus):
    weights = list()
    for mu in mus:
        if mu <= 4.685:
            weights.append((1-(mu/4.685)**2)**2)
        else:
            weights.append(0)
            
    return np.asarray(weights)

In [6]:
def MSD(X, iterations=1000):
    fit_samples = int((len(X)+2)/2)
    weights = np.ones(len(X))
    best_weights = None
    best_params = None
    indices = np.array(range(len(X)))
    
    SDs = None
    best_SDn = sys.maxsize
    for iter in range(iterations):
        # 1. Random choice subset
        sampled_indices = np.random.choice(range(len(indices)), size=fit_samples, replace=False)
        X_subset = X[sampled_indices]
        w_subset = weights[sampled_indices]
        
        # 2. Use subset to calculate weights (it can pickout some outlier at the same time)
        params = LS_circle(X_subset, w_subset)
        
        # 3.
        SDs = get_symmetry_distances(X, params, weights)
        SDn = np.sum(SDs) / len(X)
        
        # 4.
        sigma = np.median(abs(SDs - np.median(SDs))) / 0.6745
        mus = np.divide(SDs, sigma)
        weights = get_weights(mus)
        
        #  5.Evaluate how's the now weights go
        if SDn < best_SDn:
            best_Sdn = SDn
            best_weights = weights.copy()
            
    
    best_cx, best_cy, best_r, _ = LS_circle(X, best_weights)
    return np.asarray([best_cx, best_cy, best_r])
        

In [48]:
a = np.array(range(5))
type(a[0])

numpy.int64

In [64]:
# Step1. 產生測試資料

# 1-1 生成圓形資料點
num_samples = 100
theta = np.linspace(0, 2*np.pi, num_samples)
r = np.random.rand((num_samples))

x_actual = 10 * np.cos(theta)
y_actual = 10 * np.sin(theta)
x_measured = x_actual + np.random.rand(len(x_actual)) * 0.1 - 0.05
y_measured = y_actual + np.random.rand(len(x_actual)) * 0.1 - 0.05
x_measured = np.append(x_measured, np.array(range(50)) / 100.0)
y_measured = np.append(y_measured, np.zeros(50) + np.random.rand(50) * 10)

# Step2. 客製化錯誤函式 及 客製化擬合模型
X = np.transpose([x_measured, y_measured])
weights = np.ones(len(X))

cx, cy, r, _ = cf.least_squares_circle(X)
print("Standard:")
print("{}, {}, {}".format(cx, cy, r))
print()

      
params = MSD(X, iterations=200)
print("MSD:")
print(type(params))
print("{}, {}, {}".format(params[0], params[1], params[2]))

Standard:
-0.31200104609134616, -1.9306993600731857, 9.04015719685562

MSD:
<class 'numpy.ndarray'>
0.15722948215893492, -0.8985226944506666, 8.633819387199898


In [61]:
os.chdir("/home/mj/HardDisk/ARCS/img/X-Ray/Case_1/Cut")
## Read image
img, gray = read_img("10_mean_0.bmp")
height, width = get_imgInfo(img)

## Image processing
blur = cv2.GaussianBlur(gray,(5,5),0)
dst = cv2.fastNlMeansDenoising(blur,None,10,7,21)

_, threshed = cv2.threshold(dst,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

edge = edge_detect(threshed, 0, 20)
ret, labels = cv2.connectedComponents(edge)


#get edge points
edge_points = []
for i in range(height):
    for j in range(width):
        if edge[i,j] == 255:
            edge_points.append([j,i])
            
# edge_points = np.transpose(edge_points)
params = MSD(edge_points, iterations=1)

print(params)

## Show processed images    
cv2.imshow("Gray", gray)
cv2.imshow("Blur", blur)
cv2.imshow("Thresh", threshed)
# cv2.imshow("Median", median)
cv2.imshow("Edge", edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: only integer scalar arrays can be converted to a scalar index

In [9]:
os.chdir("/home/mj/HardDisk/ARCS/img/X-Ray/Case_1/Cut")
try:
    
    images = os.listdir()
    images.sort()
    
    predictions = []
    for image in images:
        img, gray = read_img(image)
        height, width = get_imgInfo(img)
        
        blur = cv2.GaussianBlur(gray,(5,5),0)
        dst = cv2.fastNlMeansDenoising(blur,None,10,7,21)
        _, threshed = cv2.threshold(dst,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#         median = median_filter(threshed, 7)
        edge = edge_detect(threshed, 0, 20)
                    
        # get edge points
        edge_points = []
        for i in range(height):
            for j in range(width):
                if edge[i,j] == 255:
                    edge_points.append([j,i])
        edge_points = np.asarray(edge_points)
        
        params = MSD(edge_points, iterations=20)
        predictions.append([params[0], params[1]])
        print("{}: {}, {}, {}".format(image, params[0], params[1], params[2]))

    
except Exception as e:
    print(e)

10_mean_0.bmp: 203.65464493261084, 99.367345882543, 66.22657417738439
10_mean_1.bmp: 203.5747241911206, 98.98953435855212, 66.34563494589469
10_mean_10.bmp: 203.738432319077, 98.85396132139716, 66.90945265576747
10_mean_11.bmp: 203.626529839616, 98.89936197558417, 66.19941564106092
10_mean_12.bmp: 203.54701002486308, 99.08990226817902, 66.7690353443038
10_mean_13.bmp: 203.5660112206098, 99.03747498957406, 66.35196536940757
10_mean_14.bmp: 203.5197168893377, 99.17047006680615, 66.86155628170687
10_mean_15.bmp: 203.461735683288, 99.10321010196355, 66.251589904635
10_mean_16.bmp: 203.58208139703407, 99.07931989439476, 67.06518325963964
10_mean_17.bmp: 203.76235784482049, 99.252596993026, 66.44853922433487
10_mean_18.bmp: 203.4429173245481, 98.90431725560505, 66.96925434933996
10_mean_19.bmp: 203.60731044187438, 98.87894994591605, 66.81839539717615
10_mean_2.bmp: 203.40426289337063, 98.93078087623586, 66.3801297078988
10_mean_20.bmp: 203.2772681189207, 99.00434341365462, 66.2557996579618
1

In [10]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (203.60084, 99.06241)
Variance     : (0.01609, 0.02107)
Max Length   : 0.54946, 0.71240 (pixels)
